In [1]:
import requests
from bs4 import BeautifulSoup

In [8]:
# !pip install requests

In [2]:
raw = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [16]:
print(raw.json)

<bound method Response.json of <Response [200]>>


In [3]:
soup = BeautifulSoup(raw.content, 'html.parser')

In [40]:
count = 0
list_row_element = []
concatenate_rows = soup.select('tbody')[0]

In [41]:
for link in concatenate_rows.find_all('td'):
    count+=1
    list_row_element.append(link.get_text().replace("\n",""))

In [42]:
count

867

In [43]:
Postcode = list_row_element[::3]
Borough = list_row_element[1::3]
Neighbourhood = list_row_element[2::3]

In [44]:
import pandas as pd

In [45]:
df = pd.DataFrame({'Postcode':Postcode, 'Borough': Borough, 'Neighbourhood': Neighbourhood})
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [46]:
df_borough = df[df['Borough'] !='Not assigned']
df_borough.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [49]:
for _, row in df_borough.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [54]:
data = df_borough.copy()
data.shape

(212, 3)

In [55]:
data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [64]:
data_final = data.groupby(('Postcode','Borough'))['Neighbourhood'].apply(lambda x: ", ".join(x))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [67]:
data_final = pd.DataFrame(data_final)
data_final.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [80]:
data_final.reset_index(level=None, inplace=True)

In [82]:
data_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [81]:
data_final.shape

(103, 3)

In [84]:
# from geopy.geocoders import Nominatim

In [87]:
# geolocator = Nominatim()
# address  = 'Downtown Toronto'
# location = geolocator.geocode(address)
# print(location.longitude)

-79.3808116451341


In [91]:
! wget https://cocl.us/Geospatial_data


--2019-02-18 14:41:56--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-18 14:41:57--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-18 14:41:57--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-18 

In [94]:
df_location = pd.read_csv('Geospatial_data')


NameError: name 'df_loaction' is not defined

In [95]:
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
data_final['Latitude'] = df_location[data_final['Postcode'] == df_location['Postal Code']]['Latitude']

In [97]:
data_final['Longitude'] = df_location[data_final['Postcode'] == df_location['Postal Code']]['Longitude']

In [98]:
data_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
